In [1]:
import pandas as pd
import py_vncorenlp
import json
import string
import os

py_vncorenlp.download_model(save_dir=os.environ["VNCORENLP"])

VnCoreNLP model folder /home/ptdat/vncorenlp already exists! Please load VnCoreNLP from this folder!


In [2]:
cwd = os.getcwd()
vncorenlp = py_vncorenlp.VnCoreNLP(save_dir=os.environ["VNCORENLP"])
os.chdir(cwd)

2024-04-27 11:50:16 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-04-27 11:50:16 INFO  PosTagger:23 - Loading POS Tagging model
2024-04-27 11:50:17 INFO  NerRecognizer:34 - Loading NER model
2024-04-27 11:50:25 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [3]:
import joblib

tokenizer = joblib.load("./tokenizer.pkl")
def tokenize(text: str):
    text = " ".join(vncorenlp.word_segment(text))
    encoding = tokenizer.encode(text)
    return encoding.tokens

In [4]:
with open("words.txt", "r") as f:
  vn_dict = f.readlines()

  vn_dict = set(map(
      lambda s: json.loads(s)["text"].lower(),
      vn_dict
  ))

In [5]:
cmts = pd.read_csv('/home/ptdat/Desktop/ViSFD/data/Train.csv').comment.values

In [6]:
vn_dict.update(
    list(string.punctuation) +
    ["game", "lag", "wifi", "samsung", "oppo", "youtube", "pubg"] +
    ["iphone", "zalo", "web", "xiaomi", "note", "vivo", "pro", "plus"] +
    ["redmi", "android", "facebook", "realme", "max", "full", "galaxy"] +
    ["bluetooth", "app", "nokia", "google", "huawei", "fps", "reno"] +
    ["vsmart", "miui", "việt nam", "tác vụ", "cuộc gọi"]
)

In [7]:
os.remove("./misspelling.txt")
with open("misspelling.txt", "w") as f:
  for cmt in cmts:
    tokens = [
        token.replace("_", " ")
        for token in tokenize(cmt)
        if not token.isnumeric()
    ]
    misspelled = [
      token
      for token in tokens
      if token not in vn_dict
    ]
    f.write("\n".join(misspelled) + "\n")